In [ ]:
!pip install datasets

In [ ]:
# import dependencies
import wandb                     # For experiment tracking
import torch                   # For tensor computations and model training
import datasets                # For working with HuggingFace datasets
import argparse                # For parsing command line arguments (if needed)
import numpy as np             # For numerical operations
import transformers            # HuggingFace Transformers library
import math

# Additional imports for our implementation
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, default_data_collator
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Define our model and dataset to use
MODEL_NAME = "ibm/MoLFormer-XL-both-10pct"
DATASET_PATH = "scikit-fingerprints/MoleculeNet_Lipophilicity"

# Define the LoRA Adapter module

In [ ]:
class LoRAAdapter(nn.Module):
    def __init__(self, hidden_size, r=4, alpha=1):
        """
        LoRA adapter that approximates a low-rank update.
        r: Rank of the low-rank update.
        alpha: Scaling factor.
        """
        super(LoRAAdapter, self).__init__()
        self.r = r
        self.alpha = alpha
        # lora_A: shape (r, hidden_size)
        # lora_B: shape (hidden_size, r)
        self.lora_A = nn.Parameter(torch.zeros(r, hidden_size))
        self.lora_B = nn.Parameter(torch.zeros(hidden_size, r))
        self.scaling = self.alpha / self.r

        # Initialize lora_A using a variant of kaiming_uniform and lora_B as zeros.
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)

    def forward(self, x):
        """
        x: Tensor of shape (batch_size, seq_length, hidden_size)
        Computes the low-rank update: (x @ lora_A^T) @ lora_B^T * scaling.
        """
        # Compute (batch, seq_length, r)
        lora_intermediate = x @ self.lora_A.t()
        # Compute (batch, seq_length, hidden_size)
        lora_update = lora_intermediate @ self.lora_B.t()
        return lora_update * self.scaling

# Define the regression model with a regression head and integrate the LoRA adapters into the transformer layers.

In [ ]:
class MoLFormerWithRegressionHead(nn.Module):
    def __init__(self, model_name):
        super(MoLFormerWithRegressionHead, self).__init__()
        # Load the pre-trained MoLFormer base model.
        self.base_model = AutoModel.from_pretrained(model_name, deterministic_eval=True, trust_remote_code=True)
        # Retrieve the hidden size from the model configuration.
        hidden_size = self.base_model.config.hidden_size
        # Define a linear layer as the regression head to map hidden states to a single output.
        self.regression_head = nn.Linear(hidden_size, 1)

        # Inject LoRA adapters into each transformer layer.
        self.inject_lora()

    def inject_lora(self):
        """
        Inject LoRA adapters into each transformer layer of the base model.
        This example assumes the transformer blocks are available as self.base_model.encoder.layer.
        """
        hidden_size = self.base_model.config.hidden_size
        if hasattr(self.base_model, 'encoder') and hasattr(self.base_model.encoder, 'layer'):
            for i, layer in enumerate(self.base_model.encoder.layer):
                # Create a LoRA adapter for the current layer.
                adapter = LoRAAdapter(hidden_size)
                # Attach the adapter to the layer.
                layer.lora_adapter = adapter
                # Save the original forward method.
                orig_forward = layer.forward

                # Define a new forward method that adds the LoRA update.
                def make_new_forward(orig_forward, adapter):
                    def new_forward(*args, **kwargs):
                        out = orig_forward(*args, **kwargs)
                        # If the output is a tuple, assume the first element is the hidden states.
                        if isinstance(out, tuple):
                            hidden = out[0]
                            adapted_hidden = hidden + adapter(hidden)
                            # Return a new tuple with the adapted hidden state and the remaining outputs unchanged.
                            return (adapted_hidden,) + out[1:]
                        else:
                            return out + adapter(out)
                    return new_forward

                # Replace the layer's forward with the new one.
                layer.forward = make_new_forward(orig_forward, adapter)
        else:
            print("Warning: Could not find encoder layers for LoRA injection.")

    def forward(self, input_ids, attention_mask=None):
        # Forward pass through the adapted base model.
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        # Extract the representation of the [CLS] token (assumed to be the first token).
        cls_representation = outputs.last_hidden_state[:, 0, :]
        # Pass the [CLS] representation through the regression head.
        regression_output = self.regression_head(cls_representation)
        return regression_output.squeeze(-1)  # Remove extra dimensions if necessary

# Tokenization function for SMILES strings

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["SMILES"],        # Use the SMILES string column.
        padding="max_length",      # Pad sequences to the maximum length.
        truncation=True,           # Truncate sequences longer than max_length.
        max_length=128,            # Maximum sequence length.
        return_attention_mask=True # Include attention masks.
    )

# Entry point

In [ ]:
if __name__ == "__main__":
    # Initialize wandb for experiment tracking (optional).
    wandb.init(project="lora_finetuning", name="lora_molformer_regression")

    # Set the device: GPU if available, else CPU.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # -----------------------------
    # Load and Prepare the Dataset
    # -----------------------------

    # Load the Lipophilicity dataset from HuggingFace.
    raw_dataset = load_dataset(DATASET_PATH)

    # Check if a 'test' split exists; if not, create one from the 'train' split.
    if "test" not in raw_dataset:
        # Create an 80/20 train/test split from the original training data.
        split_dataset = raw_dataset["train"].train_test_split(test_size=0.2, seed=42)
        # Construct a new DatasetDict with 'train' and 'test' splits.
        dataset = DatasetDict({
            "train": split_dataset["train"],
            "test": split_dataset["test"]
        })
    else:
        dataset = raw_dataset

    # Load the tokenizer associated with our MoLFormer model.
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

    # Tokenize the dataset using our tokenization function.
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Rename the label column to "labels" for consistency with our training loop.
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

    # Set the format of the dataset to PyTorch tensors.
    tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    # Create DataLoaders for the training and test sets.
    BATCH_SIZE = 16  # Adjust based on your memory constraints.
    train_loader = DataLoader(tokenized_datasets["train"], batch_size=BATCH_SIZE, shuffle=True, collate_fn=default_data_collator)
    test_loader = DataLoader(tokenized_datasets["test"], batch_size=BATCH_SIZE, shuffle=False, collate_fn=default_data_collator)


# Initialize the Regression Model with LoRA

In [ ]:
    # Create an instance of the regression model and move it to the chosen device.
    model = MoLFormerWithRegressionHead(MODEL_NAME).to(device)

    # Freeze all base model parameters except for those in the LoRA adapters.
    for name, param in model.base_model.named_parameters():
        if "lora_adapter" not in name:
            param.requires_grad = False

    # Optionally, print out which parameters are trainable.
    trainable_params = [name for name, param in model.named_parameters() if param.requires_grad]
    print("Trainable parameters:", trainable_params)

# Fine-Tune on the Regression Task

In [ ]:
    # Define the loss function (Mean Squared Error for regression).
    criterion = nn.MSELoss()
    # Create an optimizer that only updates trainable parameters.
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    num_epochs = 10  # Adjust the number of epochs as needed.

    model.train()  # Set the model to training mode.
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in train_loader:
            # Move input data and labels to the device.
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()  # Ensure labels are float.

            optimizer.zero_grad()  # Reset gradients.
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # Forward pass.
            loss = criterion(outputs, labels)  # Compute the loss.
            loss.backward()  # Backpropagate.
            optimizer.step()  # Update only trainable parameters.

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
        wandb.log({"epoch": epoch+1, "loss": avg_loss})

# Save the Fine-Tuned Model

In [ ]:
    LORA_MODEL_SAVE_PATH = "lora_molformer_regression.pth"
    torch.save(model.state_dict(), LORA_MODEL_SAVE_PATH)
    print(f"LoRA fine-tuned model saved to {LORA_MODEL_SAVE_PATH}")

# Evaluate the Model on the Test Set

In [ ]:
    model.eval()  # Set the model to evaluation mode.
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    predictions = np.array(predictions)
    actuals = np.array(actuals)
    mse = np.mean((predictions - actuals)**2)  # Compute Mean Squared Error.
    print(f"Test Mean Squared Error (MSE): {mse:.4f}")
    wandb.log({"test_mse": mse})

    # Finish the wandb run.
    wandb.finish()